![alt text][logo]

[logo]:https://wri-public-data.s3.amazonaws.com/resourcewatch/18_LOGO_ResourceWatch/18_LOGO_ResourceWatch_PB.png "Logo Title Text 2"

#Resource Watch Query Tool

In [0]:
#@title # **Pemasangan Alat Bantu** { display-mode: "form" }
#@markdown ###Tekan tombol “play” di sebelah kiri untuk memasang alat bantu bagi analisis Anda.
!pip install geojson -q
import pandas as pd
import codecs
import requests
import random
import urllib.parse
import warnings
import numpy as np
import json
import shapely.wkt
import geojson
warnings.simplefilter(action='ignore', category=FutureWarning)
!pip install LMIPy -q
import LMIPy
import sys
print('Selesai menginstal modul!')
print('Anda dapat beralih ke sel berikutnya.')

#Querying Data within a Boundary Area (Optional)

Are you interested in global data or data for a specific country or region? For global data, you can skip this section.

If you are interested in a specific country or region, please complete this section. Please note that only point data can be analyzed within a boundary area. You cannot analyze polygon data within a boundary area with this tool.

In [0]:
#@title ## Choose the Dataset that will Define Your Boundary Area
#@markdown You can type the word **national** in the 'shapefile_dataset_id' field if you would like to do a national-level analysis. You do not need to enter anything in the 'shapefile_layer_id' for a national-level analysis.
#@markdown 
#@markdown If you would like to use boundaries from a different vector dataset on Resource Watch, find the dataset that contains the boundary you want to use, navigate to its metadate page, and copy the dataset ID into the 'shapefile_dataset_id' field, below. You can use boundaries from any polygon (non-point) vector dataset on Resource Watch.
#@markdown 
#@markdown If you are interested in sub-national administrative boundaries, you may want to use the Subnational Political Boundaries dataset (http://bit.ly/2YZov2Y), which has the following dataset ID:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Politcial-Boundaries-GADM-adminitrative-level-1-1490086842541
#@markdown 
#@markdown Here are some additional example vector dataset IDs that you might want to use for your boundary area:
#@markdown *   World Database on Protected Areas (http://bit.ly/2Z5ucfE): de452a4c-a55c-464d-9037-8c3e9fe48365 
#@markdown *   Maritime Boundaries (http://bit.ly/2YYCm9L): bf5877eb-399a-4237-b510-b1d41049e3bc
#@markdown 
#@markdown Once you have entered your dataset ID, you can push the play button on the left to see a list of layers available for this dataset. Choose the layer that contains your boundary area, and enter the id in the 'shapefile_layer_id' field and push play again.
#@markdown 


#@markdown ###Please enter a dataset ID, then push the play button on the left.

shapefile_dataset_id = None
shapefile_layer_id = None
shapefile_layer_name = None
shapefile_ds_name = None
try:
  #Get vector dataset
  shapefile_dataset_id = "" #@param {type:"string"}
  shapefile_dataset_id = shapefile_dataset_id.strip()
  if shapefile_dataset_id == 'national':
    country_analysis = 'Y'
    print('Analyzing data at the national-level.')
    print('Anda dapat beralih ke sel berikutnya.')  
  else:
    country_analysis = 'N'
    if shapefile_dataset_id == ('de452a4c-a55c-464d-9037-8c3e9fe48365' or 'World-Database-on-Protected-Areas-Global-1490086842549'):
      wdpa = 'Y'
    else:
      wdpa = 'N'
    #Get name from raster dataset
    url = 'https://api.resourcewatch.org/dataset/'+shapefile_dataset_id+'?includes=layer,metadata'
    r = requests.get(url)
    for ds in r.json()['data']['attributes']['metadata']:
      if ds['attributes']['application']=='rw':
        shapefile_ds_name = ds['attributes']['name']
    print('Analyzing dataset: {}'.format(shapefile_ds_name))
    #Get layers from shapefile dataset
    print('Available layers for '+shapefile_ds_name +' dataset:')
    layer_dict = dict()
    layers = r.json()['data']['attributes']['layer']
    for layer in layers:
      if layer['attributes']['application']==['rw']:
        analysis_layer_name = layer['attributes']['name']
        analysis_layer_id =  layer['id']
        layer_dict[analysis_layer_id]=analysis_layer_name
        print('     '+analysis_layer_name+' - Layer ID: '+analysis_layer_id)
        #print('       -Layer ID: '+analysis_layer_id)
    shapefile_layer_id = ""#@param {type:"string"}
    if shapefile_layer_id=="":
      print("Please enter a layer ID from these options, and push play again.")
    else:
      try:
        shapefile_layer_id=shapefile_layer_id.strip()
        shapefile_layer_name = layer_dict[shapefile_layer_id]
        print('Analyzing layer: {}'.format(shapefile_layer_name))
        print('Anda dapat beralih ke sel berikutnya.')
      except:
        print("We couldn't find that layer ID. Please try again.")
except:
  print("We couldn't find that dataset ID. Please try again.")
  



In [0]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass
#@markdown ## **Search for Your Boundary Area by ID**
#@markdown If you are doing a national-level analysis, enter the ISO3 code for the country of interest. You will easily find the ISO3 code by googling "ISO3 code for" followed by the name of the country. In Indonesia, for example, the ISO3 code is IDN.
#@markdown
#@markdown If you are doing analysis for another shapefile dataset on Resource Watch, first, open the dataset that contains your boundary area of interest on the Resource Watch map. Click on the boundary area that you want to analyze data within, copy the Area ID from the window that pops up, and paste it in the 'area_id' field below.
#@markdown ###Please idenitfy the boundary area where you would like to summarize data by entering its name in the area_name field OR the Area ID from the interaction in the area_id field. Then, push the play button on the left.
orig_geo_id=None

url_len = 2000 #5400
def simplification_factor_picker(best_simple, best_len, SIMPLE_I, best_simple_high, best_len_high, lowest_simple, highest_simple):
  SIMPLE_I+=1
  #first we will sample a range of simplificaiton factors
  if SIMPLE_I==1:
    simplification_factor=0.01
  elif SIMPLE_I==2:
    simplification_factor=0.05
  elif SIMPLE_I==3:
    simplification_factor=0.1
  elif SIMPLE_I==4:
    simplification_factor=0.5
  #next we will narrow down the simplification factor between the best two solutions
  else:
    if not best_simple:
      simplification_factor = highest_simple + 0.1
    elif not best_simple_high:
      simplification_factor = lowest_simple - 0.001
    else:
      low_int = int(best_simple_high*10000)
      high_int = int(best_simple*10000)
      #print('generating a random number between {} and {}'.format(low_int, high_int))
      simplification_factor = random.randint(low_int,high_int)/10000
  #print('trying {}'.format(simplification_factor))
  return simplification_factor, SIMPLE_I

def simplification_algorithm(current_simple, current_len, shapefile_dataset_id, table, cartodb_id):
  SIMPLE_I = 0
  best_len = 0
  best_len_high = np.inf
  best_simple = None
  best_simple_high = None
  lowest_simple = np.inf
  highest_simple = 0
  print('Simplifying geometry...', end="")
  while SIMPLE_I<20 or (best_len<(url_len-1000) and SIMPLE_I<40):
    if SIMPLE_I <20:
      if SIMPLE_I<19:
        print('{}%...'.format(SIMPLE_I*5), end="")
      else:
        print('{}%...'.format(SIMPLE_I*5))
    elif SIMPLE_I==20:
      print('This boundary area is very complicated...simplifying further...')
    current_simple, SIMPLE_I = simplification_factor_picker(best_simple, best_len, SIMPLE_I, best_simple_high, best_len_high, lowest_simple, highest_simple)
    try:
      geo_id, feature = return_simplified_geometries(shapefile_dataset_id, table, cartodb_id, current_simple)
    except:
      continue
    if current_simple < lowest_simple:
      lowest_simple = current_simple
      #print('lowest_simple: {}'.format(lowest_simple))
    if current_simple > highest_simple:
      highest_simple = current_simple
      #print('highest_simple: {}'.format(highest_simple))
    current_len = len(str(feature))
    #print('Round {}, simplification factor: {}, length: {}'.format(SIMPLE_I, current_simple, current_len))

    if current_len < url_len and current_len > best_len:
      #print('New best feature!')
      best_len = current_len
      best_simple = current_simple
      best_geo_id = geo_id
      best_feature = feature
    elif current_len > url_len and current_len < best_len_high:
      #print('New best feature (high)!')
      best_len_high = current_len
      best_simple_high = current_simple
  return best_geo_id, best_feature, best_simple, best_len

def get_table_name(shapefile_layer_id):
  url= 'https://api.resourcewatch.org/layer/{shapefile_layer_id}'.format(shapefile_layer_id=shapefile_layer_id)
  r = requests.get(url)
  layer_sql = r.json()['data']['attributes']['layerConfig']['body']['layers'][0]['options']['sql']  
  list_of_words = layer_sql.split()
  indices = [i for i, x in enumerate(list_of_words) if x == "FROM"]
  possible_tables = []
  for idx in indices:
    next_word = list_of_words[idx + 1]
    if "select" not in next_word.lower():
      possible_tables.append(next_word)
  if len(np.unique(possible_tables))==1:
    table = np.unique(possible_tables)[0]
    carto_account = r.json()['data']['attributes']['layerConfig']['account']
  else:
    print("Sorry, we are having trouble finding the shapefiles for this layer. Please report the dataset you are searching for to amelia.snyder@wri.org so that she can fix this!")
    table = ""
  return table, carto_account

def return_geometries(table, carto_account, cartodb_id):
  headers = {
    'Content-Type': 'application/json',
  }
  params_rw=json.dumps({
      "provider":{
          "type": "carto",
          "table": table,
          "user": carto_account,
          "filter": "cartodb_id={}".format(cartodb_id)
      }
  })
  r = requests.post('https://api.resourcewatch.org/v1/geostore/', data=params_rw, headers=headers)
  geo_id=r.json()['data']['id']
  feature = json.dumps(r.json()['data']['attributes']['geojson']['features'][0]['geometry'])
  return geo_id, feature

def return_simplified_geometries(shapefile_dataset_id, table, cartodb_id, simplification_factor):
  sql_statement = "SELECT ST_AsText(ST_Simplify(the_geom, {simpl})) as the_geom FROM {table} WHERE cartodb_id={cartodb_id}".format(table=table, cartodb_id=cartodb_id, simpl=simplification_factor)
  url= 'https://api.resourcewatch.org/query/{dataset_id}?sql={sql}'.format(dataset_id=shapefile_dataset_id, sql=urllib.parse.quote(sql_statement))
  r = requests.get(url)
  geom_wkt_str=r.json()['data'][0]['the_geom']
  wkt = shapely.wkt.loads(geom_wkt_str)
  g = LMIPy.Geometry(s=wkt)
  geo_id = g.id
  feature = geojson.Feature(geometry=wkt, properties={}).geometry
  return geo_id, feature

if shapefile_dataset_id!="" and shapefile_layer_id!="":
  area_id = ""#@param {type:"string"}
  area_id = area_id.strip()
  #area_name = ""#@param {type:"string"}
  #area_name = area_name.strip()
  if area_id == "":
    print('Please enter the area ID of the area you want to analyze.')
  else:
    if country_analysis == 'Y':
      area_name = area_id
      print('Searching for {area}...'.format(area=area_name))
      try:
        url = 'https://api.resourcewatch.org/v2/geostore/admin/{iso}'.format(iso=area_name)
        r=requests.get(url)
        feature = json.dumps(r.json()['data']['attributes']['geojson']['features'][0]['geometry'])
        geo_id=r.json()['data']['id']
        print('We found it!')
        orig_geo_id = geo_id
        feature_len = len(str(feature))
        simplification_factor = 0
        if feature_len>url_len:
          table = 'wri_countries_a'
          sql_statement = "SELECT * FROM {table} WHERE iso_a3='{iso}'".format(table=table, iso=area_name)
          ds_id = '20662342-dcdd-4a42-9f58-bcc80217de71'
          url= 'https://api.resourcewatch.org/query/{dataset_id}?sql={sql}'.format(dataset_id=ds_id, sql=urllib.parse.quote(sql_statement))
          r = requests.get(url)
          df=pd.DataFrame(r.json()['data'])
          cartodb_id = df.loc[0, 'cartodb_id']
          geo_id, feature, simplification_factor, feature_len = simplification_algorithm(simplification_factor, feature_len, ds_id, table, cartodb_id)
        #print('final simplification factor: {}, final url length: {}'.format(simplification_factor, feature_len))
        if len(str(feature))<=url_len:
          print('Your boundary area has been successfully processed.')
          print('You can move on to the next cell.')
      except:
        print("We couldn't find {area}. Please check your ISO3 code and start over.".format(area=area_name))
    else:
      table, carto_account = get_table_name(shapefile_layer_id)
      if area_id!="":
        try:
          int(area_id)
        except Exception as e:
          print("Please make sure you entered the boundary area's name in the area_name field (not the area_id field).")
          raise StopExecution
        cartodb_id=area_id
        print('Searching for area ID {area} in {shp_name}...'.format(area=area_id, shp_name=shapefile_layer_name))
      elif area_name!="":
        print('Searching for {area} in {shp_name}...'.format(area=area_name, shp_name=shapefile_layer_name))
        sql_statement = "SELECT * FROM {table} LIMIT 1".format(table=table)
        url= 'https://api.resourcewatch.org/query/{dataset_id}?sql={sql}'.format(dataset_id=shapefile_dataset_id, sql=urllib.parse.quote(sql_statement))
        r = requests.get(url)
        df_full=pd.DataFrame(r.json()['data'])
        cols = df_full.columns
        cols = cols.drop('the_geom').drop('the_geom_webmercator')
        sql_statement = "SELECT {cols} FROM {table}".format(cols=", ".join(cols), table=table)
        url= 'https://api.resourcewatch.org/query/{dataset_id}?sql={sql}'.format(dataset_id=shapefile_dataset_id, sql=urllib.parse.quote(sql_statement))
        r = requests.get(url)
        df_full=pd.DataFrame(r.json()['data'])
        try:
          area_rows = df_full[df_full.eq(area_name).any(1)]
          indices = area_rows.index.tolist()
          if len(indices)>1:
            print('Please choose the cartodb_id for the area that you are interested in from the table below:')
            display(area_rows)
            cartodb_id = int(input("Which cartodb_id would you like to use?"))
          else:
            index = area_rows.index.tolist()[0]
            cartodb_id = area_rows.loc[index, 'cartodb_id']
          print('We found it!')
        except:
          print("We couldn't find that area. Check the name of your region and try again.")
          raise StopExecution
      try:
        geo_id, feature = return_geometries(table, carto_account, cartodb_id)
        orig_geo_id = geo_id
        feature_len = len(str(feature))
        simplification_factor = 0
        if feature_len>url_len:
          geo_id, feature, simplification_factor, feature_len = simplification_algorithm(simplification_factor, feature_len, shapefile_dataset_id, table, cartodb_id)
        #print('final simplification factor: {}, final url length: {}'.format(simplification_factor, feature_len))
      except:
        print('This boundary area is too complex to pull from the API. Simplifying boundary area. This may take a few minutes...')
        feature_len=999999999
        simplification_factor = 0.05
        geo_id, feature, simplification_factor, feature_len = simplification_algorithm(simplification_factor, feature_len, shapefile_dataset_id, table, cartodb_id)
        #print('final simplification factor: {}, final url length: {}'.format(simplification_factor, feature_len))
      if len(str(feature))<=url_len:
        print('Your boundary area has been successfully processed.')
        print('Anda dapat beralih ke sel berikutnya.')  

##Map the Boundary Area and Simplification

If your boundary area was simplified, you can use the next two cells to see the original boundary area compared to the simplified boundary area. Please note that the simplified boundaries may miss some data points within the region or include data points that actually fall outside of the region. You should pay close attention to data points near the edges of the boundary areas.

In [0]:
#@markdown ##Map the Original Boundary Area
#@markdown ###Push the play button on the left to see the original boundary area.
if shapefile_dataset_id=='national':
  g = LMIPy.Geometry(id_hash=orig_geo_id)
  display(g.map())
else:
  ly = LMIPy.Layer(id_hash=shapefile_layer_id)
  try:
    g = LMIPy.Geometry(id_hash=orig_geo_id)
    display(ly.map(geometry=g))
  except Exception as e:
    print("We couldn't pull the original boundary area from the API because it was too complex. If you would like to see what it looks like, /nplease view it on Resource Watch.")

In [0]:
#@markdown ##Map the Simplified Boundary Area
#@markdown ###Push the play button on the left to see the simplified boundary area that we will use for our intersection query.
if shapefile_dataset_id=='national':
  g = LMIPy.Geometry(id_hash=geo_id)
  if simplification_factor==0:
    print("We did not have to simplify your boundary area. The original area will be used for the analysis.")
  else:
    display(g.map())
else:
  ly = LMIPy.Layer(id_hash=shapefile_layer_id)
  g = LMIPy.Geometry(id_hash=geo_id)
  if simplification_factor==0:
    print("We did not have to simplify your boundary area. The original area will be used for the analysis.")
  else:
    display(ly.map(geometry=g))

#Analyze Data with an SQL Query

In [0]:
#@markdown ##Choose a Vector Dataset to Analyze
#@markdown Please enter the dataset ID for the vector dataset that you want to analyze using SQL queries. 
#@markdown 
#@markdown Here are some example vector dataset IDs:
#@markdown 
#@markdown *  Gini Index: GINI-Index
#@markdown *  Global Power Plant Database: a86d906d-9862-4783-9e30-cdb68cd808b8
#@markdown *  Fires: VIIRS-Active-Fire-Global-1490086842549
#@markdown *  Endangered Species Critical Habitats: bio001-Alliance-for-Zero-Extinction-Endangered-Species-Sites
#@markdown *  City Expansion Index: cit037-Upward-and-Outward-Expansion-Index
#@markdown 
#@markdown ###Please enter the dataset ID for the vector dataset you want to analyze, then push the play button on the left to see available columns from the dataset and their descriptions.
dataset_id = None
#layer_id = None
#layer_name = None
ds_name = None
try:
  #Get vector dataset
  dataset_id = "" #@param {type:"string"}
  dataset_id = dataset_id.strip()
  country_analysis = 'N'
  if dataset_id == ('de452a4c-a55c-464d-9037-8c3e9fe48365' or 'World-Database-on-Protected-Areas-Global-1490086842549'):
    wdpa = 'Y'
  else:
    wdpa = 'N'
  #Get name from raster dataset
  url = 'https://api.resourcewatch.org/dataset/'+dataset_id+'?includes=layer,metadata'
  r = requests.get(url)
  analysis_carto_account = r.json()['data']['attributes']['connectorUrl'].split('/')[2].split('.')[0]
  analysis_carto_table = r.json()['data']['attributes']['connectorUrl'].split('/')[4]
  for ds in r.json()['data']['attributes']['metadata']:
    if ds['attributes']['application']=='rw':
      ds_name = ds['attributes']['name']
  print('Analyzing dataset: {}'.format(ds_name))
  #pull out the columns in the actual dataset
  query = "SELECT * FROM {table} LIMIT 1".format(table=dataset_id)
  sql_query = urllib.parse.quote(query)
  url = 'https://api.resourcewatch.org/query?sql={}'.format(sql_query)
  r = requests.get(url)
  df_full=pd.DataFrame(r.json()['data'])
  cols_to_use = df_full.columns
  cols_to_drop = ['the_geom', 'the_geom_webmercator', 'cartodb_id']
  for col in cols_to_drop:
    if col in cols_to_use:
      cols_to_use = cols_to_use.drop(col)
  #get aliases and descriptions
  url = 'https://api.resourcewatch.org/dataset/'+dataset_id+'?includes=metadata'
  r = requests.get(url)
  columns = ''
  
  if r.status_code == 200:
    try:
      for ds in r.json()['data']['attributes']['metadata']:
        if ds['attributes']['application']=='rw':
          columns = ds['attributes']['columns']
      #Generate DataFrame from columns
      #This line also replaces the 'NaN' values with '-'
      columns_df = pd.DataFrame.from_dict(columns).replace(np.nan, '-', regex=True)
      aliased_columns = columns_df.columns
      for col in cols_to_use:
        if col not in aliased_columns:
          columns_df[col]=['-' for i in range(columns_df.shape[0])]
      columns_df=columns_df[cols_to_use]
      #Replace Index names with easy to understand names
      if set(columns_df.index.values) == set(['alias','description']):
        new_names = {'alias':'Plain Text Name:','description':'Column Description:'}
        columns_df.rename(index=new_names, inplace=True)
      elif set(columns_df.index.values) == set(['alias']):
        new_names = {'alias':'Plain Text Name:'}
        columns_df.rename(index=new_names, inplace=True)
      elif set(columns_df.index.values) == set(['description']):
        new_names = {'description':'Column Description:'}
        columns_df.rename(index=new_names, inplace=True)
        
      #Add a new row that has the names of the columns available to use in SQL
      sql_columns_row = dict(zip(list(columns_df), list(columns_df)))
      new_row_name = 'Columns to Use in SQL'
      sql_columns_df = pd.DataFrame(sql_columns_row, index=[new_row_name])
      #Make that row the first one
      final_df = pd.concat([sql_columns_df, columns_df], ignore_index=False)
      #Change the column names so they're blank
      final_df.columns = [''] * len(final_df.columns)
      #final_df.style.applymap('font-weight: bold',subset=pd.IndexSlice[final_df.index[final_df.index==new_row_name], :])
  
      
      #Finally display the columns!
      print('Information on available columns is shown below:')
      display(final_df)
      print('Use the information above to decide what columns are useful to you.')
      print('Anda dapat beralih ke sel berikutnya.')
      
    #Catch error if we don't know the source
    except:
      print('Something went wrong. Double check your dataset ID!')
      
  #Catch 404 Error which is usually the wrong dataset ID
  elif r.status_code == 404:
    json_err = r.json()
    for ds in json_err['errors']:
      print(ds['detail'])
      print('Double check that your dataset ID is correct!')
    
  #Catch other error that could happen!
  else:
    json_err = r.json()
    for ds in json_err['errors']:
      print(ds['detail'])
  '''
  #Get layers from shapefile dataset
  print('Available layers for '+ds_name +' dataset:')
  layer_dict = dict()
  layers = r.json()['data']['attributes']['layer']
  for layer in layers:
    if layer['attributes']['application']==['rw']:
      layer_name = layer['attributes']['name']
      layer_id =  layer['id']
      layer_dict[layer_id]=layer_name
      print('     '+layer_id + '  '+layer_name)
  layer_id = "f9029b9a-a435-4d5f-9800-7440db013ada"#add param here
  if layer_id=="":
    print("Please enter a layer ID from these options, and push play again.")
  else:
    try:
      layer_id=layer_id.strip()
      layer_name = layer_dict[layer_id]
      print('Analyzing layer: {}'.format(layer_name))
    except:
      print("We couldn't find that layer ID. Please try again.")
   '''
except:
  print("We couldn't find that dataset ID. Please try again.")

In [0]:
#@markdown ##Build Your SQL Query
#@markdown Berikut adalah tinjauan singkat SQL queries yang telah dipelajari:

#@markdown **Lihat semua atau beberapa kolom dataset dengan query dasar**: SELECT {columns} FROM {dataset_id}<br>
#@markdown **Batasi jumlah baris yang Anda lihat dari dataset**: SELECT {columns} FROM {dataset_id} LIMIT {number_of_rows}<br>
#@markdown **Memfilter data pada beberapa kolom**: SELECT {columns} FROM {table_name} WHERE {column_name} {<, <=, >, >=, =, !=} {value}<br>
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Menerapkan beberapa filter**: SELECT {columns} FROM {dataset_id} WHERE {condition1} AND {condition2} AND {condition3}<br>
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Filter waktu**: SELECT {columns} FROM {dataset_id} WHERE {datetime_column} {<, <=, >, >=, =, !=} {YYYY-MM-DD}T{HH:MI:SS}Z<br>
#@markdown **Mengurutkan dataset berdasarkan dari nilai terbesar atau terkecil**: SELECT {columns} FROM {dataset_id} ORDER BY {column_name} {ASC, DESC}<br>
#@markdown **Statistik**: SELECT {function}({column_name}) FROM {dataset_id}<br>
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Fungsi Statistik yang tersedia**: AVG(), SUM(), MIN(), MAX(), COUNT()

class StopExecution(Exception):
  def _render_traceback_(self):
    pass
#@markdown ###Enter your API query below and run this SQL code to see the result.
query ="" #@param {type:"string"}
#@markdown Check the box below if you only want to query data in a particular boundary area of interest, rather than the whole dataset.

#@markdown In order to use the boundary area intersection feature, you must complete the **Querying Data within a Boundary Area** section above. Please note that only point data (not polygons) can be analyzed with this feature.

boundary_area_intersection = False #@param {type:"boolean"}
if boundary_area_intersection == True:
  try:
    geo_query = "st_intersects(st_setsrid(ST_GeomFromGeoJSON('{geo_json}'), 4326), the_geom) ".format(geo_json=feature)
    if 'WHERE' in query:
      start = query.split('WHERE')[0]
      end = query.split('WHERE')[1]
      ending_keyword = ''
      where_ends = ['ORDER BY', 'LIMIT']
      if any(keyword  in end for keyword in where_ends):
        low_idx = np.inf
        for keyword in where_ends:
          if keyword in end:
            if end.index(keyword) < low_idx:
              low_idx = end.index(keyword)
              ending_keyword = keyword
        where_clause = end.split(ending_keyword)[0]
        ending_clause = ending_keyword+ end.split(ending_keyword)[1]
        query = start + 'WHERE ' + where_clause + ' AND ' + geo_query + ending_clause
      else:
        query = start + 'WHERE ' + end + ' AND ' + geo_query
    else:
      query =  query + ' WHERE ' + geo_query
  except Exception as e:
    if 'feature' in str(e):
      print('You have not properly defined your boundary area. Please complete the "Querying Data within a Specific Boundary Area" section above.')
      raise StopExecution
sql_query = urllib.parse.quote(query)
url = 'https://api.resourcewatch.org/query?sql={}'.format(sql_query)
print('Data Query Link: {}'.format(url))
download_url = 'https://api.resourcewatch.org/download?sql={}'.format(sql_query)


try:
  r = requests.get(url)
  if r.status_code >= 400:
    json_err = r.json()
except Exception as e:
  print('Something went wrong. Please try running this cell again.')
  print('If you continue to get this error, try limiting your request by adding a LIMIT to the end.')
  raise StopExecution
if r.status_code >= 500:
  for ds in json_err['errors']:
    print('Please check that your dataset ID is correct.')
    raise StopExecution
    #print('HTML Error: {}'.format(ds['detail']))
elif r.status_code >= 400 and r.status_code < 500:
  for ds in json_err['errors']:
    print('Please check that your dataset ID and SQL Query are correct.') 
    raise StopExecution
    #print('HTML Error: {}'.format(ds['detail']))
elif r.ok:
  df = pd.DataFrame.from_dict(r.json()['data'])
  if not df.empty:
    print('Data Download Link: {}'.format(download_url))
    print("Results of query:")
    display(df)
  else:
    print('No data points matched your query.')
else:
  print('HTML Error: {}'.format(r.status_code))
  raise StopExecution
if boundary_area_intersection and not df.empty:
  if simplification_factor>0:
    print('\nYou have used our boundary area intersection feature.\nThis tool had to simplify the boundary area to produce these results, meaning that the results are only approximate.\nIf you would like to publish these results in a news article (or anywhere else), we recommend that you use the following\nRequest Form to request results using the original boundary area instead of the simplified version that was used here.\n')
    print('     Request Form: https://docs.google.com/forms/d/e/1FAIpQLSfdhiaGTnP7x7pv3N92MhflE7qCRprBse4xB9zvvTG9Zs9G9g/viewform\n')
    print('You will be asked to provide your contact information, the SQL query that you would like us to run, and the Geo ID (below).\n')
    req_id = '{}.{}.{}.{}.{}'.format(shapefile_dataset_id, shapefile_layer_id, analysis_carto_account, analysis_carto_table, cartodb_id)
    req_id_encoded=codecs.encode(req_id.encode('utf8'), 'hex').decode('utf8')
    print('     Geo ID: {}'.format(req_id_encoded))

